In [1]:
###############################################################################
### pthon imports                                                           ###
###############################################################################

from os import path
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import re

In [2]:
###############################################################################
### local imports                                                           ###
###############################################################################

import zachquire as zaq

In [3]:
def find_file(
    file_name='path/file.txt',
    cache=False,
    cache_age=None
    ):

    if cache==False:
        return None
    
    return path.exists(file_name)

In [41]:
def get_soup(
    url='https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    file_name='soupfiles/gitsearch.txt',
    cache=False,
    cache_age=None,
    slurper='*'
):
    '''
    
    '''
    # if we already have the data, read it locally
    file_found = find_file(file_name=file_name, cache=cache, cache_age=cache_age)
    if file_found:
        with open(file_name) as f:
            return True, BeautifulSoup(f.read())

    # otherwise go fetch the data
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text)
    slurps = soup.select(slurper)
    if len(slurps) == 0:
        print('***** NO DATA RETURNED *****')
        return False, soup
    
    # save it for next time
    with open(file_name, 'w') as f:
        f.write(str(slurps[0]))
        if len(slurps)>1:
            for slurp in slurps[1:]:
#                 print(str(slurp))
                f.write('\n' + str(slurp))
        
    with open(file_name) as f:    
        soup = BeautifulSoup(f.read())
#     pd.to_csv(slurps, header=None, index=False)

    return True, soup

In [42]:
slurped, soup = get_soup(
    url='https://github.com/search?o=desc&p=2&q=advent+of+code&s=stars&type=Repositories',
    cache=True,
    slurper='.repo-list-item a',
    file_name='soupfiles/gitsearch.txt'
)

In [67]:
def soup_loop_gitsearch(
    df,
    page_beg=1,
    page_end=20,
    url='https://github.com/',
    adder_prepage='search?o=desc&',
    adder_postpage='&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    directory='soupfiles/',
    file_name='gitsearch',
    file_suffix='.txt',
    cache=False,
    slurper='.repo-list-item a',
):
    for page in range(page_beg, page_end+1):
        use_page = str(page)
        use_url = f'{url}{adder_prepage}p={use_page}{adder_postpage}'
        use_file = f'{directory}{file_name}{use_page:>02s}{file_suffix}'
        print(f'use_page: {use_page:>2s}')
        print(f'use_url: {use_url}')
        print(f'use_file: {use_file}')
        slurped, soup = get_soup(
            url = use_url,
            file_name = use_file,
            cache = cache,
            slurper = slurper,
        )
        log_file = use_file if slurped else None
        loopdict = {
            'soup': [soup], 
            'page': [page], 
            'url': [use_url], 
            'file_name': [log_file]
        }
        df = df.append(pd.DataFrame.from_dict(loopdict), ignore_index=True)
    return df
        

In [84]:
soup_df = pd.DataFrame([], columns=['soup','page','url','file_name'])
# df
soup_df = soup_loop_gitsearch(soup_df, cache=True, page_beg=1, page_end=20)

use_page:  1
use_url: https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch01.txt
use_page:  2
use_url: https://github.com/search?o=desc&p=2&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch02.txt
use_page:  3
use_url: https://github.com/search?o=desc&p=3&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch03.txt
use_page:  4
use_url: https://github.com/search?o=desc&p=4&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch04.txt
use_page:  5
use_url: https://github.com/search?o=desc&p=5&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch05.txt
use_page:  6
use_url: https://github.com/search?o=desc&p=6&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch06.txt
use_page:  7
use_url: https://github.com/search?o=desc&p=7&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch07.txt
use_page:  8
use_url: https://gith

In [85]:
soup_df

,soup,page,url,file_name
0,"<html><body><a class=""v-align-middle"" data-hyd...",1,https://github.com/search?o=desc&p=1&q=advent+...,soupfiles/gitsearch01.txt
1,"<html><body><a class=""v-align-middle"" data-hyd...",2,https://github.com/search?o=desc&p=2&q=advent+...,soupfiles/gitsearch02.txt
2,"<html><body><a class=""v-align-middle"" data-hyd...",3,https://github.com/search?o=desc&p=3&q=advent+...,soupfiles/gitsearch03.txt
3,"<html><body><a class=""v-align-middle"" data-hyd...",4,https://github.com/search?o=desc&p=4&q=advent+...,soupfiles/gitsearch04.txt
4,"<html><body><a class=""v-align-middle"" data-hyd...",5,https://github.com/search?o=desc&p=5&q=advent+...,soupfiles/gitsearch05.txt
5,"<html><body><a class=""v-align-middle"" data-hyd...",6,https://github.com/search?o=desc&p=6&q=advent+...,soupfiles/gitsearch06.txt
6,"<html><body><a class=""v-align-middle"" data-hyd...",7,https://github.com/search?o=desc&p=7&q=advent+...,soupfiles/gitsearch07.txt
7,"<html><body><a class=""v-align-middle"" data-hyd...",8,https://github.com/search?o=desc&p=8&q=advent+...,soupfiles/gitsearch08.txt
8,"<html><body><a class=""v-align-middle"" data-hyd...",9,https://github.com/search?o=desc&p=9&q=advent+...,soupfiles/gitsearch09.txt
9,"<html><body><a class=""v-align-middle"" data-hyd...",10,https://github.com/search?o=desc&p=10&q=advent...,soupfiles/gitsearch10.txt


In [86]:
print(soup_df.soup[19])

<html><body><a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":20,"per_page":10,"query":"advent of code","result_position":1,"click_id":47785268,"result":{"id":47785268,"global_relay_id":"MDEwOlJlcG9zaXRvcnk0Nzc4NTI2OA==","model_name":"Repository","url":"https://github.com/kodsnack/advent_of_code_2015"},"client_id":null,"originating_request_id":"C7F4:2C7D:5EBEA74:9ED209C:5DF038B4","originating_url":"https://github.com/search?o=desc&amp;p=20&amp;q=advent+of+code&amp;s=stars&amp;type=Repositories","referrer":null,"user_id":null}}' data-hydro-click-hmac="9da75b1cde3ca6cefec42158aacd558eb5da021c451d01da62dd3d92bfe514b2" href="/kodsnack/advent_of_code_2015">kodsnack/<em>advent</em>_<em>of</em>_<em>code</em>_2015</a>
<a href="http://adventofcode.com" rel="nofollow">http://adventofcode.com</a>
<a class="muted-link" href="/kodsnack/advent_of_code_2015/stargazers">
<svg aria-label="star" class="octicon octicon-star" height="16" role="img" ve

In [94]:
re_url = re.compile(r'"url"\:"(.+?)"')
urllist = re_url.findall(str(soup_df.soup[0]))

urllist

[]

In [96]:
def get_repos_from_gitsearch(df, column='soup', reg_text=r'"url"\:"(.+?)"'):
    git_urls = []
    re_url = re.compile(reg_text)
    for soup in soup_df.soup:
        git_urls.extend(re_url.findall(str(soup)))
    return git_urls
        

In [99]:
url_list = get_repos_from_gitsearch(soup_df)

In [101]:
len(url_list)

200

In [87]:
# souprepo = get_soup(
#     url = 'https://github.com/mfikes/advent-of-code',
#     file_name = 'soupfiles/mfikes_advent-of-code.html',
#     slurper='html',
#     cache = False
# )


In [9]:
# souprepo = get_soup(
#     url = 'https://github.com/Diggsey/aoc2018',
#     file_name = 'soupfiles/diggsey_aoc2018.html',
#     slurper='html',
#     cache = False
# )

In [10]:
# souprepo = get_soup(
#     url = 'https://github.com/mhanberg/advent-of-code-elixir-starter',
#     file_name = 'soupfiles/mhanberg_advent-of-code-elixir-starter.html',
#     slurper='html',
#     cache = False
# )

In [80]:
# Library imports for sending request, parsing HTML, and pandas
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
# Example list of URLs from GitHub
urls = [
    "https://github.com/mfikes/advent-of-code",
    "https://github.com/ryanorsinger/intro-to-machine-learning-workshop",
    'https://github.com/Diggsey/aoc2018',
    "https://github.com/kjeliasen/AdventOfCode",
    'https://github.com/mhanberg/advent-of-code-elixir-starter'
]
output = []
for url in urls:
    request = get(url)
    soup = BeautifulSoup(request.text)
    title = url.split("/")[-1]
    author = url.split("/")[-2]
    language_info = soup.select_one(".repository-lang-stats-numbers")
    languages = [language.get_text() for language in language_info.select(".lang")]
    percentages = [percentage.get_text() for percentage in language_info.select(".percent")]
    readme = soup.select_one("#readme article")
    repo_data = {}
    repo_data["author"] = author
    repo_data["title"] = title
    repo_data["readme"] = readme
    repo_data["languages"] = languages
    repo_data["language_percentages"] = percentages
    output.append(repo_data)
df = pd.DataFrame(output)
df.to_csv("soupfiles/all_repo_data.csv")

In [81]:
df

,author,language_percentages,languages,readme,title
0,mfikes,[100.0%],[Clojure],"<article class=""markdown-body entry-content p-...",advent-of-code
1,ryanorsinger,[100.0%],[Jupyter Notebook],"<article class=""markdown-body entry-content p-...",intro-to-machine-learning-workshop
2,Diggsey,[100.0%],[Rust],None,aoc2018
3,kjeliasen,"[97.8%, 2.2%]","[Jupyter Notebook, Python]","<article class=""markdown-body entry-content p-...",AdventOfCode
4,mhanberg,[100.0%],[Elixir],"<article class=""markdown-body entry-content p-...",advent-of-code-elixir-starter


In [103]:
re_repo = re.compile(r'https\://github.com/(.+?)$')
repo_list = []
repo_check = [re_repo.findall(url)[0] for url in url_list]
repo_list.extend(repo_check)
repo_list

['Bogdanp/awesome-advent-of-code',
 'BurntSushi/advent-of-code',
 'Lysxia/advent-of-coq-2018',
 'fogleman/AdventOfCode2018',
 'mstksg/advent-of-code-2018',
 'Voltara/advent2018-fast',
 'wimglenn/advent-of-code-data',
 'ChrisPenner/Advent-Of-Code-Polyglot',
 'jaksi/advent-of-other-peoples-code',
 'molyswu/hand_detection',
 'mfikes/advent-of-code',
 'dlew/aoc-2017',
 'mstksg/advent-of-code-2017',
 'kodsnack/advent_of_code_2017',
 'Diggsey/aoc2018',
 'mstksg/advent-of-code-2019',
 'mhanberg/advent-of-code-elixir-starter',
 'borkdude/advent-of-cljc',
 'BartoszMilewski/AofC2017',
 'BartoszMilewski/AoC2019',
 'zero-to-mastery/coding_challenge-24',
 'strelec/Advent-of-Scala-2015',
 'kodsnack/advent_of_code_2016',
 'narimiran/AdventOfCode2017',
 'glguy/advent2018',
 'sciyoshi/advent-of-rust-2017',
 'badouralix/advent-of-code-2018',
 'bhauman/advent-of-clojure-2016',
 'CameronAavik/AdventOfCode',
 'campoy/advent-of-code-2018',
 'gchan/advent-of-code-ruby',
 'glguy/advent2017',
 'ttencate/aoc201

In [105]:
filepath = "datafiles/data.json"
zaq.scrape_github_data(repos=repo_list, filepath=filepath)
git_json = pd.read_json(filepath)

Bogdanp/awesome-advent-of-code
https://raw.githubusercontent.com/Bogdanp/awesome-advent-of-code/master/README.md <class 'str'>
BurntSushi/advent-of-code
https://raw.githubusercontent.com/BurntSushi/advent-of-code/master/README.md <class 'str'>
Lysxia/advent-of-coq-2018
https://raw.githubusercontent.com/Lysxia/advent-of-coq-2018/master/README.md <class 'str'>
fogleman/AdventOfCode2018
https://raw.githubusercontent.com/fogleman/AdventOfCode2018/master/README.md <class 'str'>
mstksg/advent-of-code-2018
https://raw.githubusercontent.com/mstksg/advent-of-code-2018/master/README.md <class 'str'>
Voltara/advent2018-fast
https://raw.githubusercontent.com/Voltara/advent2018-fast/master/README.md <class 'str'>
wimglenn/advent-of-code-data
https://raw.githubusercontent.com/wimglenn/advent-of-code-data/master/README.rst <class 'str'>
ChrisPenner/Advent-Of-Code-Polyglot
https://raw.githubusercontent.com/ChrisPenner/Advent-Of-Code-Polyglot/master/README.md <class 'str'>
jaksi/advent-of-other-peoples

mitsuhiko/aoc19
https://raw.githubusercontent.com/mitsuhiko/aoc19/master/README.md <class 'str'>
mevdschee/aoc-stats
https://raw.githubusercontent.com/mevdschee/aoc-stats/master/README.md <class 'str'>
ttencate/aoc2016
https://raw.githubusercontent.com/ttencate/aoc2016/master/README.md <class 'str'>
willkill07/AdventOfCode2017
https://raw.githubusercontent.com/willkill07/AdventOfCode2017/master/README.md <class 'str'>
dlew/aoc-2016
https://raw.githubusercontent.com/dlew/aoc-2016/master/README.md <class 'str'>
krisajenkins/AdventOfCode
https://raw.githubusercontent.com/krisajenkins/AdventOfCode/master/README.md <class 'str'>
willkill07/AdventOfCode2016
https://raw.githubusercontent.com/willkill07/AdventOfCode2016/master/README.md <class 'str'>
encse/adventofcode
https://raw.githubusercontent.com/encse/adventofcode/master/README.md <class 'str'>
tfausak/advent-of-code
https://raw.githubusercontent.com/tfausak/advent-of-code/master/README.markdown <class 'str'>
afonsojramos/advent-of-code

nnja/advent_of_code_2018
https://raw.githubusercontent.com/nnja/advent_of_code_2018/master/README.md <class 'str'>
scy/advent-of-code
https://raw.githubusercontent.com/scy/advent-of-code/master/README.md <class 'str'>
def-/nim-advent-of-code-2015
None <class 'NoneType'>
ejhobbs/Advent-Of-Code
https://raw.githubusercontent.com/ejhobbs/Advent-Of-Code/master/README.md <class 'str'>
nickbclifford/advent-of-code-2018
https://raw.githubusercontent.com/nickbclifford/advent-of-code-2018/master/README.md <class 'str'>
1Computer1/advent
https://raw.githubusercontent.com/1Computer1/advent/master/README.md <class 'str'>
danthedaniel/Advent-of-Code-2017
https://raw.githubusercontent.com/danthedaniel/Advent-of-Code-2017/master/README.md <class 'str'>
JEG2/advent_of_code_2018
None <class 'NoneType'>
armstnp/advent-of-code-2019
None <class 'NoneType'>
rosa/advent-of-code
https://raw.githubusercontent.com/rosa/advent-of-code/master/README.md <class 'str'>
orez-/Advent-of-Code-2017
https://raw.githubuse

In [106]:
git_json

,language,readme_contents,repo
0,Python,# Awesome Advent of Code\n\nThis is a collecti...,Bogdanp/awesome-advent-of-code
1,Rust,BurntSushi's 2018 Advent of Code solutions\n==...,BurntSushi/advent-of-code
2,Coq,Advent of Code 2018 in Coq\n==================...,Lysxia/advent-of-coq-2018
3,Python,# Advent of Code 2018\n\nMy solutions to the A...,fogleman/AdventOfCode2018
4,Haskell,Advent of Code 2018\n===================\n\n*[...,mstksg/advent-of-code-2018
5,C++,# advent2018-fast\n\n[Advent of Code 2018](htt...,Voltara/advent2018-fast
6,Python,Advent of Code data\n===================\n\n|p...,wimglenn/advent-of-code-data
7,Python,Advent of Code\n==============\n\nThis is a co...,ChrisPenner/Advent-Of-Code-Polyglot
8,Python,# Advent of Other People's Code\nFetches rando...,jaksi/advent-of-other-peoples-code
9,Python,using Neural Networks (SSD) on Tensorflow.\n\n...,molyswu/hand_detection


In [16]:
git_json.to_csv('json_test.csv')